In [ ]:
import os
import cv2
import math
import numpy as np


# Threshold by which lines will be rejected wrt the horizontal
REJECT_DEGREE_TH = 4.0


In [ ]:
def FilterLines(Lines):
    FinalLines = []
    
    for Line in Lines:
        [[x1, y1, x2, y2]] = Line
        # Calculating equation of the line: y = mx + c
        if x1 != x2:
            m = (y2 - y1) / (x2 - x1)
        else:
            m = 100000000
        c = y2 - m*x2
        # theta will contain values between -90 -> +90. 
        theta = math.degrees(math.atan(m))

        # Rejecting lines of slope near to 0 degree or 90 degree and storing others
        if REJECT_DEGREE_TH <= abs(theta) <= (90 - REJECT_DEGREE_TH):
            l = math.sqrt( (y2 - y1)**2 + (x2 - x1)**2 )    # length of the line
            FinalLines.append([x1, y1, x2, y2, m, c, l])
    if len(FinalLines) > 15:
        FinalLines = sorted(FinalLines, key=lambda x: x[-1], reverse=True)
        FinalLines = FinalLines[:15]
    
    return FinalLines


In [ ]:

def GetLines(Image):
    # Converting to grayscale
    GrayImage = cv2.cvtColor(Image, cv2.COLOR_BGR2GRAY)
    # Blurring image to reduce noise.
    BlurGrayImage = cv2.GaussianBlur(GrayImage, (5, 5), 1)
    # Generating Edge image
    EdgeImage = cv2.Canny(BlurGrayImage, 40, 255)

    # Finding Lines in the image
    Lines = cv2.HoughLinesP(EdgeImage, 1, np.pi / 180, 50, 10, 15)

    # Check if lines found and exit if not.
    if Lines is None:
        print("Not enough lines found in the image for Vanishing Point detection.")
        exit(0)
    
    # Filtering Lines wrt angle
    FilteredLines = FilterLines(Lines)

    return FilteredLines


In [ ]:


def GetVanishingPoint(Lines):
    VanishingPoint = None
    MinError = 100000000000

    for i in range(len(Lines)):
        for j in range(i+1, len(Lines)):
            m1, c1 = Lines[i][4], Lines[i][5]
            m2, c2 = Lines[j][4], Lines[j][5]

            if m1 != m2:
                x0 = (c1 - c2) / (m2 - m1)
                y0 = m1 * x0 + c1

                err = 0
                for k in range(len(Lines)):
                    m, c = Lines[k][4], Lines[k][5]
                    m_ = (-1 / m)
                    c_ = y0 - m_ * x0

                    x_ = (c - c_) / (m_ - m)
                    y_ = m_ * x_ + c_

                    l = math.sqrt((y_ - y0)**2 + (x_ - x0)**2)

                    err += l**2

                err = math.sqrt(err)

                if MinError > err:
                    MinError = err
                    VanishingPoint = [x0, y0]
                
    return VanishingPoint


In [ ]:

def point_pos(x1,y1,A,B, VanishingPoint):
    detT = (B[1]-VanishingPoint[1])*(A[0]-VanishingPoint[0]) + (VanishingPoint[0]-B[0])*(A[1]-VanishingPoint[1])
    beta1 = ((B[1]-VanishingPoint[1])*(x1-VanishingPoint[0]) + (VanishingPoint[0]-B[0])*(y1-VanishingPoint[1]))/detT
    alpha1 = ((VanishingPoint[1]-A[1])*(x1-VanishingPoint[0]) + (A[0]-VanishingPoint[0])*(y1-VanishingPoint[1]))/detT
    gamma1 = 1-alpha1-beta1
    return 0<= alpha1 <=1 and 0<= beta1 <= 1 and 0<=gamma1<=1

def walldetection(Lines, V):
   # lines that exist in the area bounded by vanishing point and left corners correspond to the left wall
   # lines that exist in the area bounded by vanishing point and right corners correspond to the right corner
    A = (0,0) #left_up_corner
    B = (0,240) #left_down_corner
   
    C = (320,0) #right_up_corner
    D = (320,240) #right_down_corner
    for line in Lines:
        [[x1, y1, x2, y2]] = line
        if point_pos(x1,y1,A,B,V) and point_pos(x2,y2,A,B,V):
            wall = "LEFTTWALL"
            return wall
        elif point_pos(x1,y1,C,D,V) and point_pos(x2,y2,C,D,V):
            wall = "RIGHTWALL"
            return wall
        else: wall = "NOWALL"    
    return wall

In [7]:

if __name__ == "__main__":
        Image = cv2.imread("benchmark_data\\20231106-182012\\506_-1.999885817130722_15.978629028113254_0.png")
        #dimensions of the image - 320 * 240
        # Getting the lines form the image
        Lines = GetLines(Image)
        
        # Get vanishing point
        VanishingPoint = GetVanishingPoint(Lines)
        # Wall = walldetection(Lines,VanishingPoint)
        # print(Wall)
        # Checking if vanishing point found
        if VanishingPoint is None:
            print("Vanishing Point not found. Possible reason is that not enough lines are found in the image for determination of vanishing point.")

        # Drawing lines and vanishing point
        for Line in Lines:
            cv2.line(Image, (Line[0], Line[1]), (Line[2], Line[3]), (0, 255, 0), 2)
        cv2.circle(Image, (int(VanishingPoint[0]), int(VanishingPoint[1])), 1, (0, 0, 255), -1)
        print(VanishingPoint)
        # Showing the final image
        cv2.imshow("OutputImage", Image)
        cv2.waitKey(0)

C:\Users\vaish\AppData\Local\Temp\ipykernel_8156\2993712405.py:11: RuntimeWarning: overflow encountered in scalar multiply
  c = y2 - m*x2


ValueError: too many values to unpack (expected 1)